In [5]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

import mne
import os
from ieeg.viz.mri import plot_subj, subject_to_info, plot_on_average #this used to import plot subj sig and nonsig, and plot electrodes from mri directly, but in order to save aaron's changes, i put them into jim_mri
from src.analysis.vis.jim_mri import plot_subj_sig_and_nonsig, _plot_electrodes, get_sub_dir
from ieeg.io import raw_from_layout, get_data
# from utils import load_sig_chans, get_sig_chans_per_subject, make_sig_electrodes_per_subject_and_roi_dict, channel_names_to_indices
from src.analysis.utils.general_utils import *

import json
import pyvista as pv
pv.global_theme.allow_empty_mesh = True

['/Users/erinburns/Desktop/GlobalLocal', '/opt/anaconda3/envs/ieeg/lib/python313.zip', '/opt/anaconda3/envs/ieeg/lib/python3.13', '/opt/anaconda3/envs/ieeg/lib/python3.13/lib-dynload', '', '/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages', 'C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/']


### plot single subject's chosen electrodes

In [12]:
subject_folder='/Users/erinburns/Library/CloudStorage/Box-Box/ECoG_Recon'
test = subject_to_info('D65',subjects_dir=subject_folder) 

/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


In [9]:
subj = "D0057"
base_path = r'C:\Users\erinburns\Library\CloudStorage\Box-Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
plot_these_elecs = ['RAI2', 'RAI3', 'RAI4']
plot_these_elecs_save_name = '_'.join(plot_these_elecs)

subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj(subj_without_zeroes, subj_dir=subject_folder, picks=plot_these_elecs, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_{plot_these_elecs_save_name}.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\erinburns\\Library\\CloudStorage\\Box-Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\\freqFilt\\figs/D57_RAI2_RAI3_RAI4.png'

In [6]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']

plot multiple subjects on average brain

In [7]:
def load_sig_and_all_electrode_data_and_remove_zeroes_in_subject_names(subjects, base_path, epochs_root_file, anat_root):
    all_sig_electrodes = {}
    all_electrodes = {}

    for subj in subjects:
        # Clean up subject ID (e.g., 'D0057' -> 'D57')
        subj_without_zeroes = "D" + subj[1:].lstrip('0')

        # Get subject info, which contains all channel names
        # *** FIX: Pass the anat_root path to the subjects_dir parameter ***
        info = subject_to_info(subj_without_zeroes, subjects_dir=anat_root)
        all_subj_chan_names = info.ch_names
        
        # Store all electrode names for the subject
        all_electrodes[subj_without_zeroes] = all_subj_chan_names

        # Load significant channel names for the subject
        filename = os.path.join(base_path, subj, f'sig_chans_{subj}_{epochs_root_file}.json')
        try:
            sig_chans = load_sig_chans(filename)
            all_sig_electrodes[subj_without_zeroes] = sig_chans
        except FileNotFoundError:
            print(f"Warning: Sig chans file not found for {subj}. Skipping.")
            all_sig_electrodes[subj_without_zeroes] = []

    return all_sig_electrodes, all_electrodes

def make_indices_plottable(electrodes_dict, anat_root):
    subjects = []
    plottable_indices = []
    length = 0
    
    for subject, electrodes in electrodes_dict.items():
        # *** FIX: Pass the anat_root path to the subjects_dir parameter ***
        info = subject_to_info(subject, subjects_dir=anat_root)
        
        # Find the index for each electrode and add the offset
        for electrode in electrodes:
            if electrode in info['ch_names']:
                plottable_indices.append(length + info['ch_names'].index(electrode))
            else:
                print(f"Warning: Electrode '{electrode}' not found in {subject}")

        length += len(info['ch_names'])
        subjects.append(subject)
        
    return subjects, plottable_indices

# ===================================================================
# --- Main Script ---
# ===================================================================

HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

epochs_root_files = ["stimulus_err_corr_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"]

for epochs_root_file in epochs_root_files:
    data_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")
    save_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "vis", "figs", f'{epochs_root_file}')
    anat_path = os.path.join(LAB_root, 'ECoG_Recon_Full')

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 2. DATA LOADING
    # Pass the anat_path to your loading function
    allSigElectrodes, allElectrodes = load_sig_and_all_electrode_data_and_remove_zeroes_in_subject_names(subjects, data_path, epochs_root_file, anat_root=anat_path)

    # 3. DATA TRANSFORMATION
    # Pass the anat_path to your transformation function
    subjects_for_plot, sig_indices = make_indices_plottable(allSigElectrodes, anat_root=anat_path)
    _, all_indices = make_indices_plottable(allElectrodes, anat_root=anat_path)

    # 4. VISUALIZATION
    # Plot all electrodes first in a neutral color
    fig = plot_on_average(subjects_for_plot, picks=all_indices, rm_wm=False, hemi='both', color=(0.8, 0.8, 0.8), size=0.4)

    # Overlay the significant electrodes in a bright color
    fig = plot_on_average(subjects_for_plot, picks=sig_indices, rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4, fig=fig)

    fig.save_image(os.path.join(save_path, "sigAndAllElectrodes_for_" + str(len(subjects)) + "_subjects_" + epochs_root_file + ".png"))
    fig.close()

    # plot just the significant electrodes in a bright color
    fig2 = plot_on_average(subjects_for_plot, picks=sig_indices, rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4)
    fig2.save_image(os.path.join(save_path, "sigElectrodes_for_" + str(len(subjects)) + "_subjects_" + epochs_root_file + ".png"))
    fig2.close()

    # plot just all electrodes in a neutral color
    fig3 = plot_on_average(subjects_for_plot, picks=all_indices, rm_wm=False, hemi='both', color=(0.8, 0.8, 0.8), size=0.4)
    fig3.save_image(os.path.join(save_path, "allElectrodes_for_" + str(len(subjects)) + "_subjects_" + epochs_root_file + ".png"))
    fig3.close()

    print(f"Finished processing and saved figures to: {save_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erinburns/Library/CloudStorage/Box-Box/CoganLab/ECoG_Recon_Full/D57/elec_recon/D57_elec_locations_RAS_brainshifted.txt'